In [9]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import glob
import sarge

def curlcmd(url, database, destout):
    
    return "curl '{database}{url}' -b cookies > {destout}".format(database = database,
                                                    url = url, 
                                                    destout = destout)

In [10]:
username='img_username'
password='img_password'

database = 'https://genome.jgi.doe.gov'

In [11]:
!curl 'https://signon.jgi.doe.gov/signon/create' --data-urlencode 'login=jmb636@cornell.edu' --data-urlencode 'password=rzx4KHQ2ejw@twu_czw' -c cookies > /dev/null

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   164    0   107  100    57    288    153 --:--:-- --:--:-- --:--:--   443


In [3]:
xmls = glob.glob('../../mgdata/*2019.xml')

xml = xmls[0]

tree = ET.parse(xml)
root = tree.getroot()

mgid = os.path.basename(xml).split(".")[0]

outdir = mgid

In [4]:
for child in root:
    print(child.attrib['name'])

Sequencing QC Reports
Raw Data
QC and Genome Assembly
Metagenome Report Tables
IMG Data
Filtered Raw Data
Binning Data


In [5]:
for child in root:
    if child.attrib['name'] == 'QC and Genome Assembly':
        for c1 in child:
            for c2 in c1:
                print(c2.attrib['filename'])

pairedMapped_sorted.bam.cov
pairedMapped.sam.gz
input.corr.fastq.gz
assembly.contigs.fasta
README.txt
README.pdf


In [8]:
start_cmd = "cd /mnt/s1/projects/arcticfe/mgdata; \
curl 'https://signon.jgi.doe.gov/signon/create' --data-urlencode \
'login=jmb636@cornell.edu' --data-urlencode 'password=rzx4KHQ2ejw@twu_czw' \
-c cookies > /dev/null "


for xml in xmls:
    # get name of 
    mgid = os.path.basename(xml).split(".")[0]

    # read xml tree of sample directory system
    tree = ET.parse(xml)
    root = tree.getroot()
    outdir = mgid

    # Filtered Raw Data
    frddir = os.path.join(outdir, 'qc_and_assembly')
    keep_file_phrases = ['METAGENOME.fastq.gz']

    
        
    for child in root:
        if child.attrib['name'] == 'QC and Genome Assembly':
            for c1 in child:
                for c2 in c1:
        
                    filename = c2.attrib['filename']

                    destout = os.path.join(frddir, filename)

                    if not os.path.exists(destout) or os.path.getsize(destout) == 0:
                        url = c2.attrib['url']


                        mgcmd = curlcmd(url, database, destout)
                        full_cmd = '{};{}'.format(start_cmd, mgcmd)
                        prefix = 'qsub -q low -l walltime=5:00:00,mem=20G,ncpus=1 -j oe -o ~/out/{}_{}.out'.format(mgid,filename)
                        print('echo "{full_cmd}"|{prefix}'.format(full_cmd = full_cmd, prefix = prefix))
                        #print(full_cmd)
                        print("\n")

echo "cd /mnt/s1/projects/arcticfe/mgdata; curl 'https://signon.jgi.doe.gov/signon/create' --data-urlencode 'login=jmb636@cornell.edu' --data-urlencode 'password=rzx4KHQ2ejw@twu_czw' -c cookies > /dev/null ;curl 'https://genome.jgi.doe.gov/portal/ext-api/downloads/get_tape_file?blocking=true&url=/OKS_WetSedge1_19_3/download/_JAMO/60be329dc399d4ad32fd2b6d/pairedMapped_sorted.bam.cov' -b cookies > OKS_Pond6_19July2019/qc_and_assembly/pairedMapped_sorted.bam.cov"|qsub -q low -l walltime=5:00:00,mem=20G,ncpus=1 -j oe -o ~/out/OKS_Pond6_19July2019_pairedMapped_sorted.bam.cov.out


echo "cd /mnt/s1/projects/arcticfe/mgdata; curl 'https://signon.jgi.doe.gov/signon/create' --data-urlencode 'login=jmb636@cornell.edu' --data-urlencode 'password=rzx4KHQ2ejw@twu_czw' -c cookies > /dev/null ;curl 'https://genome.jgi.doe.gov/portal/ext-api/downloads/get_tape_file?blocking=true&url=/OKS_WetSedge1_19_3/download/_JAMO/60be329dc399d4ad32fd2b6a/pairedMapped.sam.gz' -b cookies > OKS_Pond6_19July2019/qc_an

In [ ]:
start_cmd = "cd /mnt/s1/projects/arcticfe/mgdata; \
curl 'https://signon.jgi.doe.gov/signon/create' --data-urlencode \
'login=jmb636@cornell.edu' --data-urlencode 'password=rzx4KHQ2ejw@twu_czw' \
-c cookies > /dev/null "




for xml in xmls:
    # get name of 
    mgid = os.path.basename(xml).split(".")[0]

    # read xml tree of sample directory system
    tree = ET.parse(xml)
    root = tree.getroot()
    outdir = mgid

    # Filtered Raw Data
    frddir = os.path.join(outdir, 'filtered_raw_data')
    keep_file_phrases = ['METAGENOME.fastq.gz']


    for child in root:
        if child.attrib['name'] == 'Filtered Raw Data':

            for frchild in child:
                for phrase in keep_file_phrases:
                    if phrase in frchild.attrib['filename']:
                        filename = frchild.attrib['filename']

                        destout = os.path.join(frddir, filename)
                        if os.path.getsize(destout) == 0:
                        
                            url = frchild.attrib['url']


                            mgcmd = curlcmd(url, database, destout)
                            full_cmd = '{};{}'.format(start_cmd, mgcmd)
                            prefix = 'qsub -q route -l walltime=12:00:00,mem=20G,ncpus=8 -j oe -o ~/out/{}.out'.format(mgid)
                            print('echo "{full_cmd}"|{prefix}'.format(full_cmd = full_cmd, prefix = prefix))
                            #print(full_cmd)
                            print("\n")